In [ ]:
import numpy as np
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub 

from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Mount for Drive Owner

In [ ]:
import pathlib
data_train = pathlib.Path('/content/gdrive/My Drive/Colab Notebooks/Capstone/train') 
data_test = pathlib.Path('/content/gdrive/My Drive/Colab Notebooks/Capstone/test') 

Mount for Non Owner

In [ ]:
import pathlib
data_train = pathlib.Path('/content/gdrive/My Drive/Capstone/train') 
data_test = pathlib.Path('/content/gdrive/My Drive/Capstone/test') 

In [ ]:
#preprocessing data
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
	data_train,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=126
)

test_generator = test_datagen.flow_from_directory(
	data_test,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=126
)

Found 300 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


In [ ]:


base_model = VGG16(input_shape = (150, 150, 3), 
                          include_top = False, 
                          weights = 'imagenet')


for layer in base_model.layers:
  layer.trainable = False

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
from tensorflow.keras import Model
from tensorflow.keras import layers
x = layers.Flatten()(base_model.output)
x = layers.Dense(128, activation ='relu')(x)
x = layers.Dense (3, activation = 'softmax')(x)

model = Model (base_model.input, x)

In [ ]:
model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    epochs = 10,
    validation_data = test_generator,
    verbose = 1,
    validation_steps=3
) 

Epoch 1/10
3/3 [==============================] - ETA: 0s - loss: 2.2592 - accuracy: 0.4067 

3/3 [==============================] - 108s 28s/step - loss: 2.2592 - accuracy: 0.4067 - val_loss: 1.9315 - val_accuracy: 0.4667
Epoch 2/10
3/3 [==============================] - 84s 24s/step - loss: 1.3195 - accuracy: 0.4333
Epoch 3/10
3/3 [==============================] - 85s 24s/step - loss: 1.0361 - accuracy: 0.4600
Epoch 4/10
3/3 [==============================] - 84s 23s/step - loss: 0.6898 - accuracy: 0.7300
Epoch 5/10
3/3 [==============================] - 82s 34s/step - loss: 0.6686 - accuracy: 0.7033
Epoch 6/10
3/3 [==============================] - 82s 34s/step - loss: 0.5518 - accuracy: 0.8033
Epoch 7/10
3/3 [==============================] - 82s 23s/step - loss: 0.5728 - accuracy: 0.7333
Epoch 8/10
3/3 [==============================] - 80s 33s/step - loss: 0.4499 - accuracy: 0.8567
Epoch 9/10
3/3 [==============================] - 85s 26s/step - loss: 0.4713 - accuracy: 0.8267
Epoch 10/10
3/3 [==============================] - 82s 23s/step - loss: 0.3938 - accuracy: 0.86

In [ ]:
import matplotlib.pyplot as plt

# Plot the results
acc = history.history['accuracy']

loss = history.history['loss']


epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()

In [ ]:
LUNGCARE_TEMP_MODEL = "exp_saved_model"

In [ ]:
tf.saved_model.save(model, LUNGCARE_TEMP_MODEL)

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(LUNGCARE_TEMP_MODEL)

In [ ]:
tflite_model = converter.convert()
tflite_model_file= 'converted_model.tflite'